In [ ]:
from pyspark import SparkConf, SparkContext

### mapper_dealInput
把input用 \t 分隔，回傳 link: (start, end)

In [ ]:
def mapper_dealInput(l):
    l = l.split('\t')
    return (int(l[0]), int(l[1]))

### 處理input, 計算outlink
先創一個sparkContext, 處理input得到所有link:(start, end)  

In [ ]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("pageRank")
sc = SparkContext(conf=conf)
link = sc.textFile("input.txt")
link = link.map(mapper_dealInput)

### 計算node總數
把所有被link的node（也就是在start或end出現過的node）集合起來，計算node數量和out link數量  

In [ ]:
linkSum = link.flatMap(lambda x : [(x[0], 1), (x[1], 0)])
linkSum = linkSum.reduceByKey(lambda x, y : x + y)
nodeNum = linkSum.count()

### 紀錄沒有inlink的node，map link

先找所有end node, 不在end node中的其他node就是沒有inlink的node  
把這些沒有inlink的node特別給一條 (node, node), 且value是0的link,  
避免之後做乘法的時候，沒有inlink的node消失  
再把link map成(start, (end, outlink))

In [ ]:
ends = link.values().map(lambda x : (x, 1))
no_inlink = linkSum.subtractByKey(ends).map(lambda x: (x[0], [x[0], 0]))

link = link.join(linkSum)

### 初始化R值、b值，計算matrix BM
1. 把上面計算過的node進行初始化，設成1/nodeNum
2. 設 b = 0.8
3. 根據定義，在matrix BM 裡面，根據link : (start (end, count))設定成 (j, [ i, b /count])
4. 把之前為了避免node消失而產生的no_inlink加進matrix BM裡面

In [ ]:
prevR = linkSum.map(lambda x : (x[0], 1 / nodeNum))
# matrix BM 存成 (j ,(i, value))
b = 0.8
matBM = link.map(lambda x : (x[0], [x[1][0], b / x[1][1]]))
matBM = matBM.union(no_inlink)

### 計算 r_new = BM * r_old + (1-b)/N
1. 先把初始化好的prevR join到matrix裡，產生matT
2. 再對同樣j值的pair進行相乘，保留i值，產生（i, matBM_ji * r_i）
3. 最後把同樣i值的pair相加，再加上(1-b)/N，產生（i, r_i)
4. 計算renormalize，r_i' = r_i + (1-S)/N，產生(i, r_i)
5. 重複20個iterations
6. 最後先對 i 進行sort，再取出前10名的r_i

In [ ]:
# matT = (j, ( [ i, matA_ij ], r_j ))
for i in range(20):
    matT = matBM.join(prevR)
    matT = matT.map(lambda x : (x[1][0][0], x[1][0][1] * x[1][1]))

    matT = matT.reduceByKey(lambda x, y : (x + y))
    matT = matT.map(lambda x : (x[0], x[1] + (1-b) / nodeNum))

    # renormalize
    valueSum = matT.values().sum()
    matT = matT.map(lambda x : (x[0], x[1] + float((1 - valueSum) / nodeNum)))
    prevR = matT

ansR = prevR.sortByKey()
ansR = ansR.top(10,key=lambda x : x[1] )
print(ansR)

In [ ]:
with open("Outputfile.txt", 'w') as f:
    for i in range(min(len(ansR), 10)):
        f.write("%d\t%f\n" % (ansR[i][0], round(ansR[i][1], 6)))
sc.stop()